In [1]:
import numpy as np
from pytorchnet import PytorchNet
        
class ConvBenchmark:
    def __init__(self, **bench_args):
        self.convweights_ = bench_args["convweights"]
        if "poolings" in bench_args:
            self.poolings_ = bench_args["poolings"]
        else:
            self.poolings_ = False
        
        self.networks_ = {}
        self.networks_["pytorchCNN"] = PytorchNet(self.convweights_, self.poolings_)
        # TODO: self.networks_["tensorflowCNN"] = TensorflowNet(self.convweights_, self.poolings_)
        
        
        self.input_ = np.random.random(bench_args["inputsize"]).astype(np.float32)
        self.runtimes_ = {}
        
    def computeTime(self):
        for netname, network in self.networks_.items():
            result, t = network.forward(self.input_)
            self.runtimes_[netname] = {("result", result), ("runtime", t)}
            
            print("{} took {:.3} seconds.".format(netname, t))
            
        

In [2]:
# nn.Conv2d will take in a 4D Tensor of nSamples x nChannels x Height x Width

inputsize = (2,1,32,32)
convweights = ((1, 6, 5), (6, 16, 5))
poolings = (2,2)
benchmark = ConvBenchmark(inputsize=inputsize, convweights=convweights, poolings=poolings)

In [3]:
benchmark.computeTime()

pytorchCNN took 0.0799 seconds.
